In [ ]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import Generator as Generator 
import model_definitions

### Open input file

In [ ]:
class PcPath:
    def __init__(self,physic,commPhy):
        self.physic=physic
        self.commPhy=commPhy
        


DANIELE=PcPath(r"C:\Users\dansp\OneDrive\Desktop\Tesi\Logs\OneCar_db.txt",
               r"C:\Users\dansp\OneDrive\Desktop\Tesi\Logs\PhysycAndCommand_db.txt")
PCS=[DANIELE]
PC_INDEX=0

COLUMNS_PHY=["Player", "X", "Z", "VEL_X","VEL_Z","ROT","ANG_VEL_Y","ACC_X","ACC_Z","TILE","TILE_IND","X_RELATIVE","Z_RELATIVE","TIME"]

COLUMNS_COMM_PHY=["Player", "X", "Z", "VEL_X","VEL_Z","ROT","ANG_VEL_Y","ACC_X","ACC_Z","TILE","TILE_IND","X_RELATIVE","Z_RELATIVE","TIME","MOVE_X","MOVE_Z","BREAKING"]


In [ ]:
SEQUENCE_LENGTH=20
ONE_CAR=True
COMM_PHY=True
#PATH=r"C:\Users\dansp\OneDrive\Desktop\Tesi\SCRITTURA\OneCar_tack1_db.txt"
BATCH_SIZE=256*2
FEATURES=7 
FEATURES_PREDICTED=3
DISCARD=9

if(COMM_PHY):
    PATH=PCS[PC_INDEX].commPhy
    FEATURES +=3 #adding the command to the features 
    COLUMNS=COLUMNS_COMM_PHY
else:
    PATH=PATH=PCS[PC_INDEX].physic
    COLUMNS=COLUMNS_PHY

In [ ]:
Generator.SEQUENCE_LENGTH=SEQUENCE_LENGTH
Generator.FEATURES=FEATURES
Generator.DISCARD=DISCARD
Generator.COLUMNS=COLUMNS
Generator.COMM_PHY=COMM_PHY
train,val,test=Generator.single_care_dataframe(PATH)

In [ ]:
def SubtractDF(df):
    return df.groupby("RACE",group_keys=False).apply(Generator.subtraction_columns)

def SubtractAndBuildDF(df):
    dfs=[]
    for i in range(len(df)):
        dfs.append(SubtractDF(df[i]))
        #dfs.append(df[i])
    united=dfs[0]
    for i in range(1,len(df)):
        united=pd.concat([united,dfs[i] ],ignore_index=True)
    return united

df_train= SubtractAndBuildDF(train)
df_val= SubtractAndBuildDF(val)
df_test= SubtractAndBuildDF(test)

In [ ]:
df_train.head()

In [ ]:
gen_train=Generator.DataGenerator(BATCH_SIZE,df_train,100000)
gen_val=Generator.DataGenerator(BATCH_SIZE,df_val,100000)
gen_test=Generator.DataGenerator(BATCH_SIZE,df_test,100000)


### Model definition

#### Train and print

In [ ]:
if(ONE_CAR):
    model_definitions.cars=1
else:
    model_definitions.cars=4
model_definitions.features=FEATURES
model_definitions.sequence_length=SEQUENCE_LENGTH
model_definitions.feat_pred=FEATURES_PREDICTED

### Rules Model

In [ ]:
lstm,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.01,epoch=100, dropout=0.2, cells=6 )

ev=lstm.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
lstm2,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.002,epoch=100, dropout=0.02, cells=1)

ev=lstm2.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
mlp,history=MLP_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100)

ev=mlp.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
siamese,history=LSTM_Siamese_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100, dropout=0.02, cells=2 )

ev=siamese.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

### Physichs Model

Since physichs data can be very small, Each data is multiply by 10 

In [ ]:
NAME=f"MLP_DISCARD_{DISCARD}_DELTA_LONG_TRAIN"
mlp_ph,history_mlp_ph=model_definitions.MLP_Train(gen_train,gen_val,learning_rate=0.0001,epoch=1000,verbose=1,name=NAME,path="logs")

ev=mlp_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(mlp_ph,f'saved_model/{NAME}')

In [ ]:
units=32
kernel_size=1
filters=32
normalization=False
cnn_lstm,history_cnn_lstm=model_definitions.CNN_LSTM_Train(gen_train,gen_val,learning_rate=0.0001,patience=10,name=f"CNN{filters}_{kernel_size}LSTM{units}_Norm{normalization}",
                                                           path="logs_350SequenceLength",epoch=10, dropout=0.2,units=units,kernel_size=kernel_size,
                                                           filters=filters, normalization=normalization,verbose=1
                                                          )

ev=cnn_lstm.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

tf.saved_model.save(cnn_lstm,f'saved_model/CNN_LSTM{units}_{kernel_size}_{filters}_PosAssoluta')

In [ ]:
lr =1e-4
units=64
cells=3
normalization=False
name=f"LSTM{cells}_{units}_{lr}_ROT_NO_VEL_INPUT_TRACK1_{DISCARD}"
lstm_ph,history_lstm_ph=model_definitions.LSTM_Train(
    gen_train,gen_val,learning_rate=lr,patience=6,name=name,path="track1_logs",
    epoch=20, dropout=0.2, cells=cells,units=units, normalization=normalization,verbose=1
)

ev=lstm_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
#tf.saved_model.save(lstm_ph,f'saved_model/{name}')

In [ ]:
test=0
for batch in gen_val:
    print(test)
    test+=1
test

In [ ]:
test[1].shape

In [ ]:
y_pred = lstm_ph.predict(gen_test)


In [ ]:
y_pred

In [ ]:
ev

In [ ]:
lr =1e-3
units=512
cells=8
normalization=False
name=f"LSTM{cells}_{units}_{lr}_DISCARD_{DISCARD}_DELTA"
lstm_ph,history_lstm_ph=model_definitions.LSTM_Train(gen_train,gen_val,learning_rate=lr,patience=50,name=name,path="logs",
                                                     epoch=100, dropout=0.2, cells=cells,units=units, normalization=normalization,verbose=1)

ev=lstm_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(lstm_ph,f'saved_model/{name}')

In [ ]:
tf.saved_model.save(lstm_ph,f'saved_model/{name}')

In [ ]:
model=model_definitions.MLP_Model(32)
model.load_weights("saved_model\MLP0001_PosAssoluta")

In [ ]:
model.summary()

In [ ]:
zero=gen_test[0]
toP=zero[0][0]
pred=zero[1][0]
print(toP)
print(pred)
model.predict(toP)

In [ ]:
import tf2onnx
import onnx
input_signature=(tf.TensorSpec([None,SEQUENCE_LENGTH,FEATURES],tf.float32,name='x'))
onnx_model,_=tf2onnx.converter.from_keras(model,input_signature,opset=9)

In [ ]:
races=united["RACE"].max()
print(races*0.5)
training=united.loc[united['RACE'] <= races*0.5]
training.tail(10)

In [ ]:
t=df_train["ROT"].idxmax()
df_train.iloc[t-2:t+3]